# Loading Environment Variables

In [ ]:
# Define constants
WATSONX_URL="https://us-south.ml.cloud.ibm.com"
WATSONX_API_KEY=""
PROJECT_ID=""
MODEL_ID="ibm/granite-13b-chat-v2"

# Initialize Watsonx Client

In [ ]:
import os
from ibm_watsonx_ai import APIClient, Credentials
from ibm_watsonx_ai.foundation_models.utils.enums import ModelTypes

credentials = Credentials(url=WATSONX_URL, api_key=WATSONX_API_KEY)
client = APIClient(credentials)
client.set.default_project(PROJECT_ID)

# Setting Model Parameters

In [ ]:
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames

# Set model parameters
parameters = {
    GenTextParamsMetaNames.DECODING_METHOD: "sample",
    GenTextParamsMetaNames.MAX_NEW_TOKENS: 100,
    GenTextParamsMetaNames.MIN_NEW_TOKENS: 1,
    GenTextParamsMetaNames.TEMPERATURE: 0.5,
    GenTextParamsMetaNames.TOP_K: 50,
    GenTextParamsMetaNames.TOP_P: 1,
}

# Initializing Model

In [ ]:
from ibm_watsonx_ai.foundation_models import Model

model = Model(model_id=MODEL_ID, params=parameters, credentials=credentials, project_id=PROJECT_ID)

# Initialize Prompt Manager

In [ ]:
from ibm_watsonx_ai.foundation_models.prompts import PromptTemplate, PromptTemplateManager

### Create prompt Manager ############
prompt_mgr = PromptTemplateManager(
                credentials=credentials,
                project_id=PROJECT_ID
                )


# Create Prompt Input

In [ ]:
####### Prepare Prompt #######
prompt_input = """
Write a short summarization of the JIRA Comment mentioned below for {DefectId} in one paragraph.

Comment: {AllComments}

Summary:"""

### Creates and stores a prompt template ###
def create_prompt_template():    
    
    prompt_template = PromptTemplate(
        name="Summarization of Defect Comments",
        model_id=MODEL_ID,
        input_text=prompt_input,
        input_variables=["DefectId", "AllComments"],
        model_params=parameters
    )
    promptTemplate_Summarize = prompt_mgr.store_prompt(prompt_template)
    print("Summarize Prompt Id: "+ promptTemplate_Summarize.prompt_id)
    return promptTemplate_Summarize.prompt_id

In [ ]:
prompt_mgr.unlock('4e032096-ce6c-4d11-9176-9c488644cf2d')

# Deploy the Prompt Template

In [ ]:
### Assign Prompt Id for deployment ######
prompt_template_id = create_prompt_template()

### Deploys a prompt template ###
def deploy_prompt_template(prompt_id):
    
    meta_props = {
        client.deployments.ConfigurationMetaNames.NAME: "PROMPT TEMPLATE",
        client.deployments.ConfigurationMetaNames.ONLINE: {},
        client.deployments.ConfigurationMetaNames.BASE_MODEL_ID: ModelTypes.GRANITE_13B_CHAT_V2
    }
    deployment_details = client.deployments.create(prompt_id, meta_props)
    return deployment_details["metadata"]["id"]

deploymentId = deploy_prompt_template(prompt_template_id)
print("Prompt Id: " + prompt_template_id)
print("Deployment Id: " + deploymentId)

# Assign value to Prompt Input Variable

In [ ]:
#### Assign Prompt Input Variable ########
DefectId = 'XRP-1126'
AllComments = """11/July/24 10:15 AM;john.smith;Hi [~mary.jones],
Could you please investigate the issue with the CRM integration? We're noticing a delay in data syncing, and suspect it might be related to the recent 
updates in the middleware. AMS team mentioned a possible certificate expiration issue affecting our transaction processing. Could you verify if this is 
impacting the CRM as well?

"11/July/24 11:45 AM;mary.jones;Hello [~john.smith],
I've checked the CRM environment and it appears the certificate for authentication has indeed expired, causing the sync delays. 
I'll initiate the process to renew the certificate and coordinate with AMS to update the configuration accordingly. I'll keep you updated on the progress.

11/July/24 1:00 PM;john.smith;Hi [~mary.jones],
Thank you for the update. Please keep me informed once the new certificate is generated and implemented. It's crucial to ensure minimal disruption 
to our CRM operations due to this issue.
"""

# Generate Response

In [ ]:
#### Generate Summarized Response ############

generated_response = client.deployments.generate_text(
    deployment_id=deploymentId,                
    params={
        GenTextParamsMetaNames.PROMPT_VARIABLES: {
            "DefectId": DefectId,
            "AllComments": AllComments
        }
    }
)
print(generated_response)

# List Prompt Template

In [ ]:
### List all the prompt template available #######
prompts=prompt_mgr.list()
print(prompts)